In [ ]:
# Instalations
!pip install geopandas
!pip3 install datapane

In [3]:
# basic modules
import geopandas as gpd
import pandas as pd 
import json 
import glob 
import os 
# visualizaztion modules 
from bokeh.resources import INLINE 
from bokeh.io import output_notebook, show, output_file 
from bokeh.plotting import figure 
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar 
from bokeh.palettes import brewer,mpl 
# interactive visualization models 
from bokeh.io import curdoc, output_notebook 
from bokeh.models import Slider, HoverTool 
from bokeh.layouts import widgetbox, row, column
import datapane as dp

In [4]:
# dados
df = pd.read_csv('/content/drive/My Drive/Pibiti_2019 - Notebooks (Análise e Visualizações)/drive_dados/parcela_media_ano.csv')
gdf = gpd.read_file('/content/drive/My Drive/Pibiti_2019 - Notebooks (Análise e Visualizações)/drive_dados/rideTotal2.shp')

In [7]:
gdf.groupby('NM_MUNICIP').head(1)

,NM_MUNICIP,CD_GEOCODM,geometry
0,BRASILIA,5300108,"POLYGON ((-48.11521 -15.83216, -48.11528 -15.8..."
233,BURITIS,3109303,"POLYGON ((-46.24826 -15.60454, -46.24828 -15.6..."
234,UNAI,3170404,"POLYGON ((-46.89584 -16.35869, -46.89686 -16.3..."
251,ABADIANIA,5200100,"POLYGON ((-48.40991 -16.28383, -48.41010 -16.2..."
252,AGUA FRIA DE GOIAS,5200175,"POLYGON ((-47.47221 -14.80512, -47.47340 -14.8..."
253,AGUAS LINDAS DE GOIAS,5200258,"POLYGON ((-48.21078 -15.71986, -48.21080 -15.7..."
262,ALEXANIA,5200308,"POLYGON ((-48.32024 -16.29943, -48.32102 -16.3..."
263,CABECEIRAS,5204003,"POLYGON ((-46.80500 -15.86571, -46.80620 -15.8..."
264,CIDADE OCIDENTAL,5205497,"POLYGON ((-47.67707 -16.06944, -47.67782 -16.0..."
271,COCALZINHO DE GOIAS,5205513,"POLYGON ((-48.25202 -15.65920, -48.27172 -15.6..."


In [ ]:
# Função para retornar um json
def json_data(selectYear):
    ano = selectYear
    dfYear = df[df['ANO'] == ano]
    merged = gdf.merge(dfYear, left_on ='NM_MUNICIP', right_on ='NOME_MUNICIPIO', how ='left')
    merged['Valor'] = merged.VALOR_PARCELA.fillna("N/A")
    del merged['NOME_MUNICIPIO']
    del merged['VALOR_PARCELA']
    merged_json = json.loads(merged.to_json())
    json_data = json.dumps(merged_json)
    return json_data

In [ ]:
# Bokeh + Datapane - 2013
# Input GeoJSON source that contains features for plotting.
# Select a default year to show on first display.
geosource = GeoJSONDataSource(geojson = json_data(2013))

# Define a sequential multi-hue color palette.
# I chose the Plasma theme because 10 would map to yellow :)
palette = mpl['Plasma'][10]

# Instantiate LinearColorMapper that maps numbers to a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 100, high = 250)

# Add the hovering tooltips.
hover = HoverTool(tooltips = [ ('Cidade','@NM_MUNICIP'),('Valor', '@Valor')])

# Create the color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=10,width = 500, height = 20, border_line_color=None,location = (0,0), orientation = 'horizontal')

# Create the figure object.
p = figure(title = 'Valor médio do benefício - 2013', plot_height = 600 , plot_width = 950, toolbar_location = None, tools = [hover])
# Remove the grid lines. 
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
# Increase the title font size.
p.title.text_font_size = '12pt'

# Add the patch renderer to the figure. 
# Notice that this is where you provide the Score column from the json_data and the color mapper.
p.patches('xs','ys', source = geosource,fill_color = {'field' :'Valor', 'transform' : color_mapper}, line_color = 'white', line_width = 0.5, fill_alpha =1)

# Place the color bar below the map.
p.add_layout(color_bar, 'below')

# Define the callback function.
# This is what will be called when the slider value changes.
def update_plot(attr, old, new):
    yr = slider.value
    new_data = json_data((yr))
    geosource.geojson = new_data
    p.title.text = 'Valor médio do benefício por cidade da RIDE' %yr
    
# Make a slider object.
slider = Slider(title = 'Year',start = 2013, end = 2019, step = 1, value = 2013)
slider.on_change('value', update_plot)

# Make a column layout of widgetbox (slider) and plot, and add it to the current document
layout = row(p,column(slider))
curdoc().add_root(layout)

# Show the figure.
# output_notebook(INLINE)
# show(layout)



In [ ]:
# Bokeh + Datapane - 2014
# Input GeoJSON source that contains features for plotting.
# Select a default year to show on first display.
geosource = GeoJSONDataSource(geojson = json_data(2014))

# Define a sequential multi-hue color palette.
# I chose the Plasma theme because 10 would map to yellow :)
palette = mpl['Plasma'][10]

# Instantiate LinearColorMapper that maps numbers to a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 100, high = 250)

# Add the hovering tooltips.
hover = HoverTool(tooltips = [ ('Cidade','@NM_MUNICIP'),('Valor', '@Valor')])

# Create the color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=10,width = 500, height = 20, border_line_color=None,location = (0,0), orientation = 'horizontal')

# Create the figure object.
p2 = figure(title = 'Valor médio do benefício - 2014', plot_height = 600 , plot_width = 950, toolbar_location = None, tools = [hover])
# Remove the grid lines. 
p2.xgrid.grid_line_color = None
p2.ygrid.grid_line_color = None
# Increase the title font size.
p2.title.text_font_size = '12pt'

# Add the patch renderer to the figure. 
# Notice that this is where you provide the Score column from the json_data and the color mapper.
p2.patches('xs','ys', source = geosource,fill_color = {'field' :'Valor', 'transform' : color_mapper}, line_color = 'white', line_width = 0.5, fill_alpha =1)

# Place the color bar below the map.
p2.add_layout(color_bar, 'below')

# Define the callback function.
# This is what will be called when the slider value changes.
# def update_plot(attr, old, new):
#     yr = slider.value
#     new_data = json_data((yr))
#     geosource.geojson = new_data
#     p.title.text = 'Valor médio do benefício por cidade da RIDE' %yr
    
# Make a slider object.
# slider = Slider(title = 'Year',start = 2013, end = 2019, step = 1, value = 2013)
# slider.on_change('value', update_plot)

# Make a column layout of widgetbox (slider) and plot, and add it to the current document
layout = row(p2)
curdoc().add_root(layout)

In [ ]:
# Bokeh + Datapane - 2015

geosource = GeoJSONDataSource(geojson = json_data(2015))
palette = mpl['Plasma'][10]
color_mapper = LinearColorMapper(palette = palette, low = 100, high = 250)
hover = HoverTool(tooltips = [ ('Cidade','@NM_MUNICIP'),('Valor', '@Valor')])
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=10,width = 500, height = 20, border_line_color=None,location = (0,0), orientation = 'horizontal')
p3 = figure(title = 'Valor médio do benefício - 2015', plot_height = 600 , plot_width = 950, toolbar_location = None, tools = [hover])
# Remove the grid lines. 
p3.xgrid.grid_line_color = None
p3.ygrid.grid_line_color = None
# Increase the title font size.
p3.title.text_font_size = '12pt'

# Add the patch renderer to the figure. 
# Notice that this is where you provide the Score column from the json_data and the color mapper.
p3.patches('xs','ys', source = geosource,fill_color = {'field' :'Valor', 'transform' : color_mapper}, line_color = 'white', line_width = 0.5, fill_alpha =1)

# Place the color bar below the map.
p3.add_layout(color_bar, 'below')

# Make a column layout of widgetbox (slider) and plot, and add it to the current document
layout = row(p3)
curdoc().add_root(layout)

In [ ]:
# Bokeh + Datapane - 2016

geosource = GeoJSONDataSource(geojson = json_data(2016))
palette = mpl['Plasma'][10]
color_mapper = LinearColorMapper(palette = palette, low = 100, high = 250)
hover = HoverTool(tooltips = [ ('Cidade','@NM_MUNICIP'),('Valor', '@Valor')])
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=10,width = 500, height = 20, border_line_color=None,location = (0,0), orientation = 'horizontal')
p4 = figure(title = 'Valor médio do benefício - 2016', plot_height = 600 , plot_width = 950, toolbar_location = None, tools = [hover])
# Remove the grid lines. 
p4.xgrid.grid_line_color = None
p4.ygrid.grid_line_color = None
# Increase the title font size.
p4.title.text_font_size = '12pt'

# Add the patch renderer to the figure. 
# Notice that this is where you provide the Score column from the json_data and the color mapper.
p4.patches('xs','ys', source = geosource,fill_color = {'field' :'Valor', 'transform' : color_mapper}, line_color = 'white', line_width = 0.5, fill_alpha =1)

# Place the color bar below the map.
p4.add_layout(color_bar, 'below')

# Make a column layout of widgetbox (slider) and plot, and add it to the current document
layout = row(p4)
curdoc().add_root(layout)

In [ ]:
# Bokeh + Datapane - 2017

geosource = GeoJSONDataSource(geojson = json_data(2017))
palette = mpl['Plasma'][10]
color_mapper = LinearColorMapper(palette = palette, low = 100, high = 250)
hover = HoverTool(tooltips = [ ('Cidade','@NM_MUNICIP'),('Valor', '@Valor')])
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=10,width = 500, height = 20, border_line_color=None,location = (0,0), orientation = 'horizontal')
p5 = figure(title = 'Valor médio do benefício - 2017', plot_height = 600 , plot_width = 950, toolbar_location = None, tools = [hover])
# Remove the grid lines. 
p5.xgrid.grid_line_color = None
p5.ygrid.grid_line_color = None
# Increase the title font size.
p5.title.text_font_size = '12pt'

# Add the patch renderer to the figure. 
# Notice that this is where you provide the Score column from the json_data and the color mapper.
p5.patches('xs','ys', source = geosource,fill_color = {'field' :'Valor', 'transform' : color_mapper}, line_color = 'white', line_width = 0.5, fill_alpha =1)

# Place the color bar below the map.
p5.add_layout(color_bar, 'below')

# Make a column layout of widgetbox (slider) and plot, and add it to the current document
layout = row(p5)
curdoc().add_root(layout)

In [ ]:
# Bokeh + Datapane - 2018

geosource = GeoJSONDataSource(geojson = json_data(2018))
palette = mpl['Plasma'][10]
color_mapper = LinearColorMapper(palette = palette, low = 100, high = 250)
hover = HoverTool(tooltips = [ ('Cidade','@NM_MUNICIP'),('Valor', '@Valor')])
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=10,width = 500, height = 20, border_line_color=None,location = (0,0), orientation = 'horizontal')
p6 = figure(title = 'Valor médio do benefício - 2018', plot_height = 600 , plot_width = 950, toolbar_location = None, tools = [hover])
# Remove the grid lines. 
p6.xgrid.grid_line_color = None
p6.ygrid.grid_line_color = None
# Increase the title font size.
p6.title.text_font_size = '12pt'

# Add the patch renderer to the figure. 
# Notice that this is where you provide the Score column from the json_data and the color mapper.
p6.patches('xs','ys', source = geosource,fill_color = {'field' :'Valor', 'transform' : color_mapper}, line_color = 'white', line_width = 0.5, fill_alpha =1)

# Place the color bar below the map.
p6.add_layout(color_bar, 'below')

# Make a column layout of widgetbox (slider) and plot, and add it to the current document
layout = row(p6)
curdoc().add_root(layout)

In [ ]:
# Bokeh + Datapane - 2019

geosource = GeoJSONDataSource(geojson = json_data(2019))
palette = mpl['Plasma'][10]
color_mapper = LinearColorMapper(palette = palette, low = 100, high = 250)
hover = HoverTool(tooltips = [ ('Cidade','@NM_MUNICIP'),('Valor', '@Valor')])
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=10,width = 500, height = 20, border_line_color=None,location = (0,0), orientation = 'horizontal')
p7 = figure(title = 'Valor médio do benefício - 2019', plot_height = 600 , plot_width = 950, toolbar_location = None, tools = [hover])
# Remove the grid lines. 
p7.xgrid.grid_line_color = None
p7.ygrid.grid_line_color = None
# Increase the title font size.
p7.title.text_font_size = '12pt'

# Add the patch renderer to the figure. 
# Notice that this is where you provide the Score column from the json_data and the color mapper.
p7.patches('xs','ys', source = geosource,fill_color = {'field' :'Valor', 'transform' : color_mapper}, line_color = 'white', line_width = 0.5, fill_alpha =1)

# Place the color bar below the map.
p7.add_layout(color_bar, 'below')

# Make a column layout of widgetbox (slider) and plot, and add it to the current document
layout = row(p7)
curdoc().add_root(layout)

In [ ]:

# DATAPANE

# Show the figure.
r = dp.Report(

    dp.Markdown('## Valor médio do benefício por ano'), #add description to the report
    dp.Plot(p), #create a chart  
    dp.Plot(p2), #create a chart
    dp.Plot(p3), #create a chart
    dp.Plot(p4), #create a chart
    dp.Plot(p5), #create a chart
    dp.Plot(p6), #create a chart
    dp.Plot(p7), #create a chart
    dp.Markdown('## Tabela com os valores recebidos'), #dados
    dp.Table(df)
  
)

#Publish your report. Make sure to have visibility='PUBLIC' if you want to share your report
#r.publish(name='stock_report', visibility='PUBLIC')
r.save(path='relatorio.html')




In [ ]:
!datapane login --server=https://datapane.com/ --token=9034b88d6f6ad54400fceb9c06c91ed208852dd0

Logged in to https://datapane.com/ as marcos
